In [1]:
import pandas as pd
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import urllib
import pathlib
import datetime


In [2]:
CHROMEDRIVER_PATH = 'C:\\Users\\ramit\\fliprobo\\chromedriver.exe'
WINDOW_SIZE = "1920,1080"

In [3]:
# Comman function for init driver
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    pathlib.Path('download').mkdir(parents=True, exist_ok=True) 
    
    preferences = {
        "download.default_directory": os.getcwd() + os.path.sep + '/download',
    }

    chrome_options.add_experimental_option('prefs', preferences)

    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    driver.get(url)
    return driver

In [4]:
def wait_for_page_load(xpath_url):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        pass
    except NoSuchElementException:
        print("Element not found")

# 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [7]:
product_name = input('What do you want to search ? ')

What do you want to search ? guitar


In [8]:
driver = init_driver('https://www.amazon.in/')

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\3784387491.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)


In [9]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]').send_keys(product_name)

# hit search 
driver.find_element_by_xpath('//input[@id="nav-search-submit-button"]').click()

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\3160546009.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]').send_keys(product_name)
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\3160546009.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@id="nav-search-submit-button"]').click()


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [10]:
page_count = 0 
all_urls = []   
while page_count < 3:
    sleep(5)
    # wait for page load
    wait_for_page_load('//div[@class="a-section a-spacing-medium"]')    

    # finding all boxes 
    all_box = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-medium"]')

    
    # Ittrating over box to find details
    for one_box in all_box:

        try:
            url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
            all_urls.append(url)
        except:
            pass

        
    # Go to next page
    driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
    try:
        driver.find_element_by_xpath('//li[@class="a-last"]').click()
    except:
        driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
    page_count += 1
    

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2706053945.py:9: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_box = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-medium"]')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2706053945.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2706053945.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2706053945.py:25: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath

In [11]:
final_data = []
for one_url in all_urls:
    driver.get(one_url)
    
    try:
        sleep(5)
        wait_for_page_load('//span[@id="priceblock_ourprice"]')    


        try:
            brand_name = driver.find_element_by_xpath('//div[@id="productOverview_feature_div"]//span[@class="a-size-base"]').tex
        except:
            brand_name = driver.find_element_by_xpath('//a[@id="bylineInfo"]').text

        product_name = driver.find_element_by_xpath('//h1[@id="title"]').text
        try:
            rating = driver.find_element_by_xpath('//span[@id="acrPopover"]//span//span').get_attribute('innerHTML')
            ratings_count = driver.find_element_by_xpath('//span[@id="acrCustomerReviewText"]').text
        except:
            rating = '-'
            ratings_count = '-'

        try:
            price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]').text
        except:
            try:
                price = driver.find_element_by_xpath('//span[@id="priceblock_dealprice"]').text
            except:
                price = '-'


        return_exchange = driver.find_element_by_xpath('//div[@id="RETURNS_POLICY"]').text
        try:
            expected_delivery = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text
        except:
            expected_delivery = '-'
        availability = driver.find_element_by_xpath('//div[@id="availability_feature_div"]').text
        other_details = driver.find_element_by_xpath('//div[@id="feature-bullets"]').text
        product_url = one_url

        final_data.append({
            'brand_name': brand_name,
            'product_name': product_name,
            'rating': rating,
            'ratings_count': ratings_count,
            'price': price,
            'return_exchange': return_exchange,
            'expected_delivery': expected_delivery,
            'availability': availability,
            'other_details': other_details,
            'product_url': product_url,
        })
    except:
        print('Got error in: ',one_url)
        
data = pd.DataFrame(final_data)
data

""


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [12]:
keywords = ['fruits', 'cars', 'Machine Learning']

In [13]:
def download_image(all_urls):
    for url in all_urls:
        driver.get(url)
        driver.execute_script(f'a = document.createElement("a");a.setAttribute("download","");a.href = "{url}";document.body.appendChild(a);a.click();')

In [14]:
for key in keywords:
    driver.get('https://images.google.com/')
    driver.find_element_by_xpath('//form//input').send_keys(key)
    driver.find_element_by_xpath('//form//input').send_keys(Keys.ENTER)
    
    
    all_src_list = []
    
    while True:
        all_image_box = driver.find_elements_by_xpath('//div[@id="islmp"]//div[@jsname="DeysSe"]/img')
        
        for one_box in all_image_box:
            try:
                one_box.click()
                sleep(1)
                src = driver.find_element_by_xpath('//div[@id="islsp"]//a/img[contains(@src,"https://")]').get_attribute('src')
                all_src_list.append(src)
            except:
                pass

        driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
        if len(all_src_list) > 100:
            break
        
    download_image(all_src_list)


C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4143508674.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//form//input').send_keys(key)
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4143508674.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//form//input').send_keys(Keys.ENTER)
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4143508674.py:10: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_image_box = driver.find_elements_by_xpath('//div[@id="islmp"]//div[@jsname="DeysSe"]/img')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4143508674.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  src = driver.find_element_by_xpath('//di

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1 st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [15]:
driver.get('https://www.flipkart.com/')

In [16]:
# close login popup 
try:
    wait_for_page_load('//button[text()="✕"]')
    driver.find_element_by_xpath('//button[text()="✕"]').click()
except:
    pass

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\3682520032.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//button[text()="✕"]').click()


In [17]:
driver.find_element_by_xpath('//input[@name="q"]').send_keys('pixel 4A')
driver.find_element_by_xpath('//button[@type="submit"]').click()

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2764712933.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@name="q"]').send_keys('pixel 4A')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2764712933.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//button[@type="submit"]').click()


In [18]:
wait_for_page_load('//div[contains(@data-id,"MOB")]')    

# finding top all boxes 
all_box = driver.find_elements_by_xpath('//div[contains(@data-id,"MOB")]')

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\101071676.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_box = driver.find_elements_by_xpath('//div[contains(@data-id,"MOB")]')


In [19]:
all_urls = [] 
for one_box in all_box:
    all_urls.append(one_box.find_element_by_xpath('.//a').get_attribute('href'))

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\481374510.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  all_urls.append(one_box.find_element_by_xpath('.//a').get_attribute('href'))


In [20]:
final_data = []
for one_url in all_urls:
    driver.get(one_url)
    
    try:
        sleep(5)
        
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()

        wait_for_page_load('//h1')    


        brand_name = driver.find_element_by_xpath('//h1').text
        smartphone_name = driver.find_element_by_xpath('//h1').text
        colour = driver.find_element_by_xpath('//td[text()="Color"]/parent::*//li').text
        RAM = driver.find_element_by_xpath('//td[text()="RAM"]/parent::*//li').text
        storage_ROM = driver.find_element_by_xpath('//td[text()="Internal Storage"]/parent::*//li').text
        primary_camera = driver.find_element_by_xpath('//td[text()="Primary Camera"]/parent::*//li').text

        try:    
            secondary_camera = driver.find_element_by_xpath('//td[text()="Secondary Camera"]/parent::*//li').text
        except:
            secondary_camera = '-'

        try:    
            display = driver.find_element_by_xpath('//td[text()="Display Type"]/parent::*//li').text
        except:
            display = '-'

        display_size = driver.find_element_by_xpath('//td[text()="Display Size"]/parent::*//li').text

        resolution = driver.find_element_by_xpath('//td[text()="Resolution"]/parent::*//li').text

        try:
            processor = driver.find_element_by_xpath('//td[text()="Processor Type"]/parent::*//li').text
        except:
            processor = '-'

        try:
            processor_cores = driver.find_element_by_xpath('//td[text()="Processor Core"]/parent::*//li').text
        except:
            processor = '-'

        battery_capacity = driver.find_element_by_xpath('//td[text()="Battery Capacity"]/parent::*//li').text

        price = driver.find_element_by_xpath('//div[@class="CEmiEU"]').text
        product_url = one_url






        final_data.append({
            'brand_name':brand_name,
            'smartphone_name':smartphone_name,
            'colour':colour,
            'RAM':RAM,
            'storage_ROM':storage_ROM,
            'primary_camera':primary_camera,
            'secondary_camera':secondary_camera,
            'display_size':display_size,
            'display':display,
            'resolution':resolution,
            'processor':processor,
            'processor_cores':processor_cores,
            'battery_capacity':battery_capacity,
            'price':price,
            'product_url':product_url,
        })
        
    except:
        print('Got error in: ',one_url)
        
data = pd.DataFrame(final_data)
data

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2725009536.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2725009536.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  brand_name = driver.find_element_by_xpath('//h1').text
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2725009536.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  smartphone_name = driver.find_element_by_xpath('//h1').text
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2725009536.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  colour = driver.find_element_by_xpath('//td[text()="Color"]/parent::*//li')

,brand_name,smartphone_name,colour,RAM,storage_ROM,primary_camera,secondary_camera,display_size,display,resolution,processor,processor_cores,battery_capacity,price,product_url
0,"Google Pixel 4a (Just Black, 128 GB) (6 GB RAM)","Google Pixel 4a (Just Black, 128 GB) (6 GB RAM)",Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),Full HD+ OLED Display,2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,"vivo T1 44W (Starry Sky, 128 GB) (4 GB RAM)","vivo T1 44W (Starry Sky, 128 GB) (4 GB RAM)",Starry Sky,4 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),Full HD+ AMOLED Display,2400 x 1080 Pixel,Qualcomm Snapdragon 680,Octa Core,5000 mAh,"₹14,499₹19,99027% off",https://www.flipkart.com/vivo-t1-44w-starry-sk...
2,"vivo T1 44W (Ice Dawn, 128 GB) (6 GB RAM)","vivo T1 44W (Ice Dawn, 128 GB) (6 GB RAM)",Ice Dawn,6 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),Full HD+ AMOLED Display,2400 x 1080 Pixel,Qualcomm Snapdragon 680,Octa Core,5000 mAh,"₹15,999₹20,99023% off",https://www.flipkart.com/vivo-t1-44w-ice-dawn-...
3,"vivo T1 44W (Ice Dawn, 128 GB) (4 GB RAM)","vivo T1 44W (Ice Dawn, 128 GB) (4 GB RAM)",Ice Dawn,4 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),Full HD+ AMOLED Display,2400 x 1080 Pixel,Qualcomm Snapdragon 680,Octa Core,5000 mAh,"₹14,499₹19,99027% off",https://www.flipkart.com/vivo-t1-44w-ice-dawn-...
4,"vivo T1 44W (Starry Sky, 128 GB) (6 GB RAM)","vivo T1 44W (Starry Sky, 128 GB) (6 GB RAM)",Starry Sky,6 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),Full HD+ AMOLED Display,2400 x 1080 Pixel,Qualcomm Snapdragon 680,Octa Core,5000 mAh,"₹15,999₹20,99023% off",https://www.flipkart.com/vivo-t1-44w-starry-sk...
5,"vivo T1 44W (Midnight Galaxy, 128 GB) (4 GB RAM)","vivo T1 44W (Midnight Galaxy, 128 GB) (4 GB RAM)",Midnight Galaxy,4 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),Full HD+ AMOLED Display,2400 x 1080 Pixel,Qualcomm Snapdragon 680,Octa Core,5000 mAh,"₹14,499₹19,99027% off",https://www.flipkart.com/vivo-t1-44w-midnight-...
6,"vivo T1 44W (Midnight Galaxy, 128 GB) (6 GB RAM)","vivo T1 44W (Midnight Galaxy, 128 GB) (6 GB RAM)",Midnight Galaxy,6 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),Full HD+ AMOLED Display,2400 x 1080 Pixel,Qualcomm Snapdragon 680,Octa Core,5000 mAh,"₹15,999₹20,99023% off",https://www.flipkart.com/vivo-t1-44w-midnight-...
7,"Infinix Smart 6 (Light Sea Green, 64 GB) (2 G...","Infinix Smart 6 (Light Sea Green, 64 GB) (2 G...",Light Sea Green,2 GB,64 GB,8 MP + Depth Lens,5MP Front Camera,16.76 cm (6.6 inch),HD+ LCD IPS Display,1600 x 720 Pixel,Mediatek Helio A22,Quad Core,5000 mAh,"₹7,299₹8,99918% off",https://www.flipkart.com/infinix-smart-6-light...
8,"Infinix Smart 6 (Starry Purple, 64 GB) (2 GB ...","Infinix Smart 6 (Starry Purple, 64 GB) (2 GB ...",Starry Purple,2 GB,64 GB,8 MP + Depth Lens,5MP Front Camera,16.76 cm (6.6 inch),HD+ LCD IPS Display,1600 x 720 Pixel,Mediatek Helio A22,Quad Core,5000 mAh,"₹7,299₹8,99918% off",https://www.flipkart.com/infinix-smart-6-starr...
9,"Infinix Smart 6 (Heart Of Ocean, 64 GB) (2 GB...","Infinix Smart 6 (Heart Of Ocean, 64 GB) (2 GB...",Heart Of Ocean,2 GB,64 GB,8 MP + Depth Lens,5MP Front Camera,16.76 cm (6.6 inch),HD+ LCD IPS Display,1600 x 720 Pixel,Mediatek Helio A22,Quad Core,5000 mAh,"₹7,299₹8,99918% off",https://www.flipkart.com/infinix-smart-6-heart...


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [21]:
driver.get('https://www.google.com/maps/')

In [22]:
city = input('Enter city: ')

Enter city: Bicholim


In [23]:
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(city)
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(Keys.CONTROL + Keys.ENTER)

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\3010301255.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(city)
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\3010301255.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(Keys.CONTROL + Keys.ENTER)


In [24]:
current_url = driver.current_url

In [25]:
latitude, longitude, _ = current_url.split('@')[1].split('/')[0].split(',')

In [26]:
print('latitude: ',latitude)
print('longitude: ',longitude)

latitude:  15.5955562
longitude:  73.9309678


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [27]:
driver.get('https://trak.in/india-startup-funding-investment-2015/')

In [28]:
all_page_length = driver.find_elements_by_xpath('//div[@class="dataTables_length"]//select')

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2343679091.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_page_length = driver.find_elements_by_xpath('//div[@class="dataTables_length"]//select')


In [29]:
for one_page_length in all_page_length:
    select = Select(one_page_length)
    select.select_by_visible_text('100')

In [30]:
all_table = pd.read_html(driver.page_source)

In [31]:
df = pd.concat(all_table)
df

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
0,1,01/04/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,Innoven Capital,Series F,460000000,NaN
1,2,05/04/2021,Meesho,E-commerce,Online reselling platform,Bengaluru,SoftBank Vision Fund 2,Series E,300000000,NaN
2,3,14/04/2021,Swiggy,Online Food Delivery,Online Food Delivery,Bengaluru,"Amansa Holdings, Carmignac, Falcon Edge Capita...",Series J,343000000,NaN
3,4,07/04/2021,Groww,FinTech,Investment platform,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series D,83000000,NaN
4,5,14/04/2021,Beldara,E-commerce,Global B2B marketplace,Mumbai,Hindustan Media Ventures,Venture,7400000,NaN
...,...,...,...,...,...,...,...,...,...,...
31,32,26/07/2018,TheCapitalNet,Fin-Tech,Unified Global Investments Ecosystem,Hyderabad,Lindwall Family Investments LLC (LFI),Private Equity,500000.0,NaN
32,33,30/07/2018,Shuttl,Consumer Internet,Bus Aggregation Platform,Gurugram,Amazon Alexa Fund & Dentsu Ventures,Private Equity,11000000.0,NaN
33,34,30/07/2018,Cure Fit,Consumer Internet,Health and fitness Platform,Bengaluru,"IDG Ventures, Accel Partners, Kalaari Capital ...",Private Equity,120000000.0,NaN
34,35,31/07/2018,Five Star Group,Fin-Tech,Non-Bank Lending Platform For SME,Chennai,"TPG, Norwest Venture Partners, Sequoia Capital...",Private Equity,100000000.0,NaN


In [32]:
df['Date (dd/mm/yyyy)'] = pd.to_datetime(df['Date (dd/mm/yyyy)'],errors='coerce')

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2198673618.py:1: UserWarning: Parsing '14/04/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date (dd/mm/yyyy)'] = pd.to_datetime(df['Date (dd/mm/yyyy)'],errors='coerce')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2198673618.py:1: UserWarning: Parsing '31/03/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date (dd/mm/yyyy)'] = pd.to_datetime(df['Date (dd/mm/yyyy)'],errors='coerce')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2198673618.py:1: UserWarning: Parsing '30/03/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date (dd/mm/yyyy)'] = pd.to_datetime(df['Date (dd/mm/yyyy)'],errors='coerce')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2198673618.py:1: UserWarning: Parsing '23/03/2021' in DD/MM/YYYY format. Provide format or specify inf

In [33]:
df.dtypes

Sr. No.                        object
Date (dd/mm/yyyy)      datetime64[ns]
Startup Name                   object
Industry / Vertical            object
Sub-Vertical                   object
City / Location                object
Investors' Name                object
Investment Type                object
Amount (In USD)                object
Unnamed: 9                     object
dtype: object

In [34]:
df

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
0,1,2021-01-04,BYJU’S,Edu-tech,Online tutoring,Bengaluru,Innoven Capital,Series F,460000000,NaN
1,2,2021-05-04,Meesho,E-commerce,Online reselling platform,Bengaluru,SoftBank Vision Fund 2,Series E,300000000,NaN
2,3,2021-04-14,Swiggy,Online Food Delivery,Online Food Delivery,Bengaluru,"Amansa Holdings, Carmignac, Falcon Edge Capita...",Series J,343000000,NaN
3,4,2021-07-04,Groww,FinTech,Investment platform,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series D,83000000,NaN
4,5,2021-04-14,Beldara,E-commerce,Global B2B marketplace,Mumbai,Hindustan Media Ventures,Venture,7400000,NaN
...,...,...,...,...,...,...,...,...,...,...
31,32,2018-07-26,TheCapitalNet,Fin-Tech,Unified Global Investments Ecosystem,Hyderabad,Lindwall Family Investments LLC (LFI),Private Equity,500000.0,NaN
32,33,2018-07-30,Shuttl,Consumer Internet,Bus Aggregation Platform,Gurugram,Amazon Alexa Fund & Dentsu Ventures,Private Equity,11000000.0,NaN
33,34,2018-07-30,Cure Fit,Consumer Internet,Health and fitness Platform,Bengaluru,"IDG Ventures, Accel Partners, Kalaari Capital ...",Private Equity,120000000.0,NaN
34,35,2018-07-31,Five Star Group,Fin-Tech,Non-Bank Lending Platform For SME,Chennai,"TPG, Norwest Venture Partners, Sequoia Capital...",Private Equity,100000000.0,NaN


In [35]:
start_time = datetime.datetime(2020, 7, 1)
end_time = datetime.datetime(2020, 9, 30)

In [36]:
df = df[(start_time < df['Date (dd/mm/yyyy)']) & (df['Date (dd/mm/yyyy)'] < end_time )].sort_values('Date (dd/mm/yyyy)')
df

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
3,4,2020-07-04,DeHaat,AgriTech,AgriTech,Patna,"Sequoia Capital India, FMO, Omnivore and AgFun...",Series A,12000000,NaN
8,9,2020-07-08,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,16200000,NaN
5,6,2020-07-09,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000",NaN
6,7,2020-07-09,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,100000000,NaN
9,10,2020-07-13,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,500000,NaN
4,5,2020-07-13,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,3300000,NaN
8,9,2020-07-14,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,200000,NaN
3,4,2020-07-14,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,1700000,NaN
0,1,2020-07-15,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,1200000000,NaN
7,8,2020-07-15,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,292800,NaN


In [37]:
df.reset_index()

,index,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
0,3,4,2020-07-04,DeHaat,AgriTech,AgriTech,Patna,"Sequoia Capital India, FMO, Omnivore and AgFun...",Series A,12000000,NaN
1,8,9,2020-07-08,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,16200000,NaN
2,5,6,2020-07-09,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000",NaN
3,6,7,2020-07-09,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,100000000,NaN
4,9,10,2020-07-13,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,500000,NaN
5,4,5,2020-07-13,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,3300000,NaN
6,8,9,2020-07-14,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,200000,NaN
7,3,4,2020-07-14,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,1700000,NaN
8,0,1,2020-07-15,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,1200000000,NaN
9,7,8,2020-07-15,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,292800,NaN


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [38]:
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')

In [39]:
all_laptop_title = driver.find_elements_by_xpath('//div[@class="TopNumbeHeading sticky-footer"]/a')

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\628579816.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_laptop_title = driver.find_elements_by_xpath('//div[@class="TopNumbeHeading sticky-footer"]/a')


In [40]:
all_name = []
for one_title in all_laptop_title:
    all_name.append(one_title.text)
    
name_series = pd.Series(all_name)
name_series

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2566941525.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  name_series = pd.Series(all_name)


Series([], dtype: float64)

In [41]:
all_specs = driver.find_elements_by_xpath('//div[@class="Spcs-details"]')

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2308807098.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_specs = driver.find_elements_by_xpath('//div[@class="Spcs-details"]')


In [42]:
all_spec_list = []
for one_spec in all_specs:

    os_name = one_spec.find_element_by_xpath('.//td[text()="OS"]/parent::*/td[3]').text
    display = one_spec.find_element_by_xpath('.//td[text()="Display"]/parent::*/td[3]').text
    processor = one_spec.find_element_by_xpath('.//td[text()="Processor"]/parent::*/td[3]').text
    memory = one_spec.find_element_by_xpath('.//td[text()="Memory"]/parent::*/td[3]').text
   # weight = one_spec.find_element_by_xpath('.//td[text()="Weight"]/parent::*/td[3]').text
    dimension = one_spec.find_element_by_xpath('.//td[text()="Dimension"]/parent::*/td[3]').text
    graphics_processor = one_spec.find_element_by_xpath('.//td[text()="Graphics Processor"]/parent::*/td[3]').text
    
    all_spec_list.append({
        'os' : os_name,
        'display' : display,
        'processor' : processor,
        'memory' : memory,
        #'weight' : weight,
        'dimension' : dimension,
        'graphics_processor' : graphics_processor,
    })
    
df = pd.DataFrame(all_spec_list)
df

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4275476716.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  os_name = one_spec.find_element_by_xpath('.//td[text()="OS"]/parent::*/td[3]').text
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4275476716.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  display = one_spec.find_element_by_xpath('.//td[text()="Display"]/parent::*/td[3]').text
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4275476716.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  processor = one_spec.find_element_by_xpath('.//td[text()="Processor"]/parent::*/td[3]').text
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\4275476716.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  me

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//td[text()="Dimension"]/parent::*/td[3]"}
  (Session info: chrome=103.0.5060.53)
Stacktrace:
Backtrace:
	Ordinal0 [0x010D56B3+2184883]
	Ordinal0 [0x0106E5F1+1762801]
	Ordinal0 [0x00F83DA8+802216]
	Ordinal0 [0x00FB1B40+990016]
	Ordinal0 [0x00FB1DDB+990683]
	Ordinal0 [0x00FA8331+951089]
	Ordinal0 [0x00FCCB54+1100628]
	Ordinal0 [0x00FA82B4+950964]
	Ordinal0 [0x00FCCD24+1101092]
	Ordinal0 [0x00FDCF52+1167186]
	Ordinal0 [0x00FCC926+1100070]
	Ordinal0 [0x00FA6EA0+945824]
	Ordinal0 [0x00FA7D96+949654]
	GetHandleVerifier [0x01374192+2704034]
	GetHandleVerifier [0x0136687D+2648461]
	GetHandleVerifier [0x0116119A+529066]
	GetHandleVerifier [0x01160006+524566]
	Ordinal0 [0x01074F2B+1789739]
	Ordinal0 [0x01079978+1808760]
	Ordinal0 [0x01079A65+1808997]
	Ordinal0 [0x01082A01+1845761]
	BaseThreadInitThunk [0x75DFFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77597A9E+286]
	RtlGetAppContainerNa

In [43]:
df['name'] = name_series
df

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9,name
3,4,2020-07-04,DeHaat,AgriTech,AgriTech,Patna,"Sequoia Capital India, FMO, Omnivore and AgFun...",Series A,12000000,NaN,NaN
8,9,2020-07-08,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,16200000,NaN,NaN
5,6,2020-07-09,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000",NaN,NaN
6,7,2020-07-09,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,100000000,NaN,NaN
9,10,2020-07-13,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,500000,NaN,NaN
4,5,2020-07-13,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,3300000,NaN,NaN
8,9,2020-07-14,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,200000,NaN,NaN
3,4,2020-07-14,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,1700000,NaN,NaN
0,1,2020-07-15,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,1200000000,NaN,NaN
7,8,2020-07-15,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,292800,NaN,NaN


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [44]:
driver.get('https://www.forbes.com/billionaires/')

In [45]:
all_list = []

while True:

    all_boxes = driver.find_elements_by_xpath('//div[@class="table-body"]/div/div[contains(@class,"table-row ")]')
    for one_box in all_boxes:

        rank = one_box.find_element_by_xpath('.//div[@class="rank"]').text
        name = one_box.find_element_by_xpath('.//div[@class="personName"]').text
        net_worth = one_box.find_element_by_xpath('.//div[@class="netWorth"]').text
        age = one_box.find_element_by_xpath('.//div[@class="age"]').text
        citizenship = one_box.find_element_by_xpath('.//div[@class="countryOfCitizenship"]').text
        source = one_box.find_element_by_xpath('.//div[@class="source"]').text
        industry = one_box.find_element_by_xpath('.//div[@class="category"]').text

        all_list.append({
            'rank': rank,
            'name': name,
            'net_worth': net_worth,
            'age': age,
            'citizenship': citizenship,
            'source': source,
            'industry': industry,
        })
        
          
    if len(driver.find_elements_by_xpath('//button[contains(@class,"pagination-btn--next pagination-btn--disabled")]')) == 0:
        # we have more pages go to next
        driver.find_element_by_xpath('//button[contains(@class,"pagination-btn--next")]').click()
    else:
        # we are at last page
        break
        
        
df = pd.DataFrame(all_list)
df

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2269474574.py:5: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_boxes = driver.find_elements_by_xpath('//div[@class="table-body"]/div/div[contains(@class,"table-row ")]')
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2269474574.py:27: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  if len(driver.find_elements_by_xpath('//button[contains(@class,"pagination-btn--next pagination-btn--disabled")]')) == 0:
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2269474574.py:29: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//button[contains(@class,"pagination-btn--next")]').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@class,"pagination-btn--next")]"}
  (Session info: chrome=103.0.5060.53)
Stacktrace:
Backtrace:
	Ordinal0 [0x010D56B3+2184883]
	Ordinal0 [0x0106E5F1+1762801]
	Ordinal0 [0x00F83DA8+802216]
	Ordinal0 [0x00FB1B40+990016]
	Ordinal0 [0x00FB1DDB+990683]
	Ordinal0 [0x00FDEC32+1174578]
	Ordinal0 [0x00FCCB54+1100628]
	Ordinal0 [0x00FDCF52+1167186]
	Ordinal0 [0x00FCC926+1100070]
	Ordinal0 [0x00FA6EA0+945824]
	Ordinal0 [0x00FA7D96+949654]
	GetHandleVerifier [0x01374192+2704034]
	GetHandleVerifier [0x0136687D+2648461]
	GetHandleVerifier [0x0116119A+529066]
	GetHandleVerifier [0x01160006+524566]
	Ordinal0 [0x01074F2B+1789739]
	Ordinal0 [0x01079978+1808760]
	Ordinal0 [0x01079A65+1808997]
	Ordinal0 [0x01082A01+1845761]
	BaseThreadInitThunk [0x75DFFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77597A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77597A6E+238]


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [46]:
driver.get('https://www.youtube.com/watch?v=4X6_PvhGOTY')

In [47]:
wait_for_page_load('//ytd-comment-thread-renderer')

# Loading all comments
i = 1 
while True:
    i += 500
    driver.execute_script(f"window.scrollTo(0, {i});")
        
    all_box = driver.find_elements_by_xpath('//ytd-comment-thread-renderer')
    
    sleep(1)
    
    if len(all_box) > 500:
        break

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\2752078063.py:9: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  all_box = driver.find_elements_by_xpath('//ytd-comment-thread-renderer')


In [48]:
all_list = []
for one_box in all_box:
    comment = one_box.find_element_by_xpath('.//ytd-expander[@id="expander"]').text
    upvote = one_box.find_element_by_xpath('.//span[@id="vote-count-middle"]').text
    time = one_box.find_element_by_xpath('.//div[@id="header-author"]//yt-formatted-string/a').text
    
    all_list.append({
        'comment' : comment,
        'upvote' : upvote,
        'time' : time
    })
    
df = pd.DataFrame(all_list)
df

C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\482584847.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  comment = one_box.find_element_by_xpath('.//ytd-expander[@id="expander"]').text
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\482584847.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  upvote = one_box.find_element_by_xpath('.//span[@id="vote-count-middle"]').text
C:\Users\ramit\AppData\Local\Temp\ipykernel_9072\482584847.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  time = one_box.find_element_by_xpath('.//div[@id="header-author"]//yt-formatted-string/a').text


,comment,upvote,time
0,Mean while : मैं कयों मान लूँ गलती मेरी ही है।...,11K,5 days ago
1,People in Other countries use camra to prevent...,7.9K,4 days ago (edited)
2,Some Words for him : This man is literally bor...,5K,4 days ago
3,कोई धक्का-मुक्की नहीं करेगा \nसब को हसने का मौ...,3.8K,4 days ago
4,"wow, how confident she was and immediately pla...",95,22 hours ago
...,...,...,...
511,Women,,1 day ago
512,Full video upload karna bhai maaza ayenge,,1 day ago
513,Her husband be like: e shaitan saali fir bachg...,,9 hours ago
514,,,7 hours ago
